In [1]:
import os
import math
import glob
import requests
import numpy as np
import pandas as pd
#from tqdm import tqdm
from io import StringIO
from tqdm import tqdm_notebook as tqdm

In [2]:
molecule = []
iso_slug = []
isotopologue = []
path_mol_iso = []
for line in open('./data/url/api__urls.txt'):
    molecule.append(line.split('/')[-4])
    iso_slug.append(line.split('/')[-3])
    isotopologue.append(line.split('/')[-2])
    path_mol_iso.append(line.split('/')[-4] + '/' + line.split('/')[-3]
                        + '/' + line.split('/')[-2])

molecule_list = list(set(molecule))
molecule_list.sort(key=molecule.index)

iso_slug_list = list(set(iso_slug))
iso_slug_list.sort(key=iso_slug.index)

isotopologue_list = list(set(isotopologue))
isotopologue_list.sort(key=isotopologue.index)

path_mol_iso_list = list(set(path_mol_iso))
path_mol_iso_list.sort(key=path_mol_iso.index)

#print('Molecule:', molecule_list)
#print('Iso-slug:', iso_slug_list)
#print('Isotopologue:', isotopologue_list)
print('Total:', path_mol_iso_list)


Total: ['AlH/27Al-1H/AlHambra', 'C2H2/12C2-1H2/aCeTY', 'C2/12C2/8states', 'CO2/12C-16O2/UCL-4000', 'H2O/1H2-16O/POKAZATEL', 'H3O_p/1H3-16O_p/eXeL', 'NH3/14N-1H3/CoYuTe', 'TiO/46Ti-16O/Toto', 'TiO/47Ti-16O/Toto', 'TiO/48Ti-16O/Toto', 'TiO/49Ti-16O/Toto', 'TiO/50Ti-16O/Toto']


In [3]:
unc_formula = pd.DataFrame(eval(str(iso_slug_list).replace('1H','H')
                                .replace('-','').replace('_p','+')))
unc_formula.columns = ['exomol formula']
unc_formula

,exomol formula
0,27AlH
1,12C2H2
2,12C2
3,12C16O2
4,H216O
5,H316O+
6,14NH3
7,46Ti16O
8,47Ti16O
9,48Ti16O


In [4]:
path_mol_iso_list = ['NH3/14N-1H3/CoYuTe']
hitran_online = pd.DataFrame()
hitran_online['molecule ID'] = ['11']
hitran_online['isotopologue ID'] = ['1']
hitran_online['exomol formula'] = ['14NH3']
hitran_online['fractional abundance'] = ['0.995872']
hitran_online['Q(296K)'] = ['1725.22']
hitran_online


,molecule ID,isotopologue ID,exomol formula,fractional abundance,Q(296K)
0,11,1,14NH3,0.995872,1725.22


In [5]:
path_mol_iso = path_mol_iso_list[0]
M_mol_iso = 'M of ' + path_mol_iso.replace('/','__')
molecule_id = int(hitran_online['molecule ID'][0])
isotopologue_id = int(hitran_online['isotopologue ID'][0])
fractional_abundance = float(hitran_online['fractional abundance'][0])

In [6]:
read_path = './data/www.exomol.com/db/'

# Create a folder for saving result files.
# If the folder exists, save files directory,otherwise, create it.
result_path = './data/result/'
# Determine whether the folder exists or not.
if os.path.exists(result_path):
    pass
else:
    # Create the folder.
    os.makedirs(result_path, exist_ok=True)

In [7]:
path_mol_iso = path_mol_iso_list[0]
def_path = glob.glob('./data/def/' + '*' + path_mol_iso.split('/')[1]
                     + '__' + path_mol_iso.split('/')[2] + '.def')
def_reader = pd.read_csv(def_path[0], sep='\\s+', names=['1','2','3','4','5'], header=None)
list(def_reader[def_reader['4'].isin(['label'])]['1'].values)

['+/-',
 'Gamma',
 'N(Block)',
 'n1',
 'n2',
 'n3',
 'n4',
 'l3',
 'l4',
 'tau(inv)',
 'J',
 'K',
 'tau(rot)',
 'v1',
 'v2',
 'v3',
 'v4',
 'v5',
 'v6',
 'Gamma(vib)']

In [8]:
states_col_name = (['ID','energy','g_tot','J_tot','Unc']
                   + ['+/-','Gtot','i','n1','n2','n3','n4','l3',
                      'l4','taui','J','K','Grot','v1','v2','v3',
                      'v4','v5','v6','Gvib','Ecalc'])

In [9]:
s_df = dict()
states_df = pd.DataFrame()
states_filenames = glob.glob(read_path + path_mol_iso + '/' + path_mol_iso.split('/')[1]
                             + '__' + path_mol_iso.split('/')[2] + '.states.bz2')

for states_filename in states_filenames:
    s_df[states_filename] = pd.read_csv(states_filename, compression='bz2', sep='\s+',
                                        header=None, names=states_col_name,
                                        chunksize=100_000_000, iterator=True,
                                        low_memory=False)
    for chunk in s_df[states_filename]:
        states_df = states_df.append(chunk)
        
# Extract rows of states file whose uncertainty indices are small than 0.001.
unc_states_df = states_df[states_df['Unc'] < float(0.001)]

In [10]:
unc_states_df

,ID,energy,g_tot,J_tot,Unc,+/-,Gtot,i,n1,n2,...,K,Grot,v1,v2,v3,v4,v5,v6,Gvib,Ecalc
0,1,0.000000,0,0,0.000001,+,1,1,0,0,...,0,1,0,0,0,0,0,0,1,0.000000
3248,3249,1626.274716,6,0,0.000293,+,3,1,0,0,...,0,1,0,0,0,0,1,0,3,1626.274403
3251,3252,3240.162973,6,0,0.000530,+,3,4,0,0,...,0,1,0,0,0,0,2,0,3,3240.161923
3252,3253,3443.627880,6,0,0.000543,+,3,5,0,0,...,0,1,0,1,0,0,0,0,3,3443.629525
3255,3256,4416.915053,6,0,0.000090,+,3,8,0,1,...,0,1,0,1,0,0,0,2,3,4416.915050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3234483,3234484,4764.802572,270,22,0.000974,-,6,16,0,0,...,5,6,0,0,0,0,0,0,1,4764.636367
3323728,3323729,4734.322578,282,23,0.000934,+,3,10,0,0,...,13,6,0,0,0,0,0,1,5,4734.223900
3381663,3381664,4815.113744,564,23,0.000963,-,5,5,0,0,...,12,1,0,0,0,0,0,1,5,4814.998628
3381665,3381666,5015.347214,564,23,0.000971,-,5,7,0,0,...,9,5,0,0,0,0,0,0,1,5015.189140


In [11]:
pf_col_name = ['T', 'Q']
pf_path = read_path + path_mol_iso + '/' + path_mol_iso.replace('/','__') + '_pf.csv'
'''
pf_url = ('http://www.exomol.com/db/' + path_mol_iso + '/'
          + path_mol_iso.split('/')[1] + '__' + path_mol_iso.split('/')[2] + '.pf')   
response = requests.get(pf_url)
content = response.text  
pf_data = pd.read_csv(StringIO(content), sep='\s+', names=pf_col_name, header=None, engine='python')
pf_data.to_csv(pf_path, header=False)
'''
pf_df = pd.read_csv(pf_path, header=None, names=pf_col_name)
# Partition function defined as sum over states at standard 296K
Q = pf_df.iloc[296-1]['Q']
Q

1725.2795

In [12]:
T = 296                      # Reference temperature k
h = 6.62607015e-34           # Planck's const (J s)
c = 299792458                # Velocity of light (m s-1)
kB = 1.380649e-23            # Boltzmann's const (J K-1)

c2 = h * c * 100 / kB                  # Second radiation constant (cm K)
pi_c_8 = 1 / (8 * np.pi * c * 100)     # 8 * pi * c (cm-1 s)
c2_T = c2 / T                          # c2 / T  (cm)


In [13]:
def extract_unc_trans(trans_df):
    """
    
    
    """
    upper_id = trans_df['i'].values
    lower_id = trans_df['f'].values
    state_id = unc_states_df['ID'].values
    
    # Extract the same upper states ID from states_df
    unc_trans_i_df = pd.DataFrame()
    for id_1 in tqdm(state_id):
        unc_trans_i_df = unc_trans_i_df.append(trans_df[trans_df['i'].isin([id_1])])
        
    # Extract the same lower states ID from states_df
    unc_trans_df = pd.DataFrame()
    for id_2 in tqdm(state_id):
        unc_trans_df = unc_trans_df.append(unc_trans_i_df[unc_trans_i_df['f'].isin([id_2])])
        
    return unc_trans_df

In [14]:
def calculate_csv(unc_states_df, unc_trans_df):
    """
    
    
    """
    unc_upper_id = unc_trans_df['i'].values
    unc_lower_id = unc_trans_df['f'].values 
    state_id = unc_states_df['ID']
    unc_trans_num = unc_trans_df['i'].count()

    wavenumber = []                         # Vacuum wavenumber (cm−1)
    intensity = pd.DataFrame()              # Intensities (cm-1/molecule cm-2) at standard 296K         
    A_coefficient = []                      # Einstein A-coefficient
    lower_state_energy = pd.DataFrame()     # lower state energy
    uncertainty = []                        # Uncertainty indices
    weight_upper_state = pd.DataFrame()     # Statistical weight of upper state
    weight_lower_state = pd.DataFrame()     # Statistical weight of lower state
    upper_global_quanta = []                # Upper-state 'global' quanta
    lower_global_quanta = []                # Lower-state 'global' quanta
    upper_local_quanta = []                 # Upper-state 'local' quanta
    lower_local_quanta = []                 # Lower-state 'local' quanta


    for i in tqdm(range(unc_trans_num)):
        id_i = unc_upper_id[i]
        id_f = unc_lower_id[i]
        A = unc_trans_df['A_if'].values[i]                               # Einstein-A coefficient (s−1)
        g_i = unc_states_df[state_id.isin([id_i])]['g_tot'].values       # Total degeneracy of upper state
        g_f = unc_states_df[state_id.isin([id_f])]['g_tot'].values       # Total degeneracy of lower state
        E_i = unc_states_df[state_id.isin([id_i])]['energy'].values      # Upper state energy
        E_f = unc_states_df[state_id.isin([id_f])]['energy'].values      # Lower state energy
        unc_i = unc_states_df[state_id.isin([id_i])]['Unc'].values       # Uncertainty indices of upper state
        unc_f = unc_states_df[state_id.isin([id_f])]['Unc'].values       # Uncertainty indices of lower state
        
        u_n1 = unc_states_df[state_id.isin([id_i])]['n1'].values[0]
        u_n2 = unc_states_df[state_id.isin([id_i])]['n2'].values[0]
        u_n3 = unc_states_df[state_id.isin([id_i])]['n3'].values[0]
        u_n4 = unc_states_df[state_id.isin([id_i])]['n4'].values[0]
        u_l3 = unc_states_df[state_id.isin([id_i])]['l3'].values[0]
        u_l4 = unc_states_df[state_id.isin([id_i])]['l4'].values[0]
        u_J = unc_states_df[state_id.isin([id_i])]['J'].values[0]
        u_K = unc_states_df[state_id.isin([id_i])]['K'].values[0]
        u_taui = (str(unc_states_df[state_id.isin([id_i])]['taui'].values[0])
                  .replace('0','s').replace('1','a'))
        u_Gvib = (str(unc_states_df[state_id.isin([id_i])]['Gvib'].values[0])
                  .replace("1","A1'").replace("2","A2'").replace("3","E'")
                  .replace('4','A1"').replace('5','A2"').replace('6','E"'))
        u_Gtot = (str(unc_states_df[state_id.isin([id_i])]['Gtot'].values[0])
                  .replace("1","A1'").replace("2","A2'").replace("3","E'")
                  .replace('4','A1"').replace('5','A2"').replace('6','E"'))
        u_Grot = (str(unc_states_df[state_id.isin([id_i])]['Grot'].values[0])
                  .replace("1","A1'").replace("2","A2'").replace("3","E'")
                  .replace('4','A1"').replace('5','A2"').replace('6','E"'))

        l_n1 = unc_states_df[state_id.isin([id_f])]['n1'].values[0]
        l_n2 = unc_states_df[state_id.isin([id_f])]['n2'].values[0]
        l_n3 = unc_states_df[state_id.isin([id_f])]['n3'].values[0]
        l_n4 = unc_states_df[state_id.isin([id_f])]['n4'].values[0]
        l_l3 = unc_states_df[state_id.isin([id_f])]['l3'].values[0]
        l_l4 = unc_states_df[state_id.isin([id_f])]['l4'].values[0]
        l_J = unc_states_df[state_id.isin([id_f])]['J'].values[0]
        l_K = unc_states_df[state_id.isin([id_f])]['K'].values[0]
        l_taui = (str(unc_states_df[state_id.isin([id_f])]['taui'].values[0])
                  .replace('0','s').replace('1','a'))
        l_Gvib = (str(unc_states_df[state_id.isin([id_f])]['Gvib'].values[0])
                  .replace("1","A1'").replace("2","A2'").replace("3","E'")
                  .replace('4','A1"').replace('5','A2"').replace('6','E"'))
        l_Gtot = (str(unc_states_df[state_id.isin([id_f])]['Gtot'].values[0])
                  .replace("1","A1'").replace("2","A2'").replace("3","E'")
                  .replace('4','A1"').replace('5','A2"').replace('6','E"'))
        l_Grot = (str(unc_states_df[state_id.isin([id_f])]['Grot'].values[0])
                  .replace("1","A1'").replace("2","A2'").replace("3","E'")
                  .replace('4','A1"').replace('5','A2"').replace('6','E"'))

        V_i = ('%2d%2d%2d%2d%2d%2d%3s%3s' %
               (u_n1,u_n2,u_n3,u_n4,u_l3,u_l4,u_taui,u_Gvib) + ',')    # Upper-state 'global' quanta
        V_f = ('%2d%2d%2d%2d%2d%2d%3s%3s' %
               (l_n1,l_n2,l_n3,l_n4,l_l3,l_l4,l_taui,l_Gvib) + ',')    # Lower-state 'global' quanta
        Q_i = '%3s%3d%3d%3s' % (u_Gtot,u_J,u_K,u_Grot) + ','           # Upper-state 'local' quanta
        Q_f = '%3s%3d%3d%3s' % (l_Gtot,l_J,l_K,l_Grot) + ','           # Lower-state 'local' quanta

        unc = math.sqrt(unc_i ** 2 + unc_f ** 2)                       # Uncertainty idices
        v = float(abs(E_i - E_f))                                      # Vacuum wavenumber (cm−1)
        S = g_i * A * np.exp(- c2_T * E_f) * (1 - np.exp(- c2_T * v)) * pi_c_8 / (v ** 2) / Q    # Intensities

        wavenumber.append(v)
        intensity = intensity.append(pd.DataFrame(S))
        A_coefficient.append(A)
        lower_state_energy = lower_state_energy.append(pd.DataFrame(E_f))
        uncertainty.append(unc)
        weight_upper_state = weight_upper_state.append(pd.DataFrame(g_i))
        weight_lower_state = weight_lower_state.append(pd.DataFrame(g_f))
        upper_global_quanta += V_i.split(',')
        lower_global_quanta += V_f.split(',')
        upper_local_quanta += Q_i.split(',')
        lower_local_quanta += Q_f.split(',')
    
    iso_csv_df = pd.DataFrame()
    iso_csv_df['v'] = wavenumber                       # Vacuum wavenumber (cm−1)
    iso_csv_df['S'] = intensity.values                 # Intensities (cm-1/molecule cm-2) at standard 296K  
    iso_csv_df['A'] = A_coefficient                    # Einstein A-coefficient
    iso_csv_df['E_f'] = lower_state_energy.values      # Lower state energy
    iso_csv_df['Ierr'] = uncertainty                   # Uncertainty indices
    iso_csv_df['g_i'] = weight_upper_state.values      # Statistical weight of upper state
    iso_csv_df['g_f'] = weight_lower_state.values      # Statistical weight of lower state
    iso_csv_df[M_mol_iso] = molecule_id                # Molecule number
    iso_csv_df['I'] = isotopologue_id                  # Isotopologue number
    iso_csv_df['gm_a'] = np.nan                        # Air-broadened half-width
    iso_csv_df['gm_s'] = np.nan                        # Self-broadened half-width
    iso_csv_df['n_a'] = np.nan                         # Temperature-dependence exponent for gamma_air
    iso_csv_df['dt_a'] = np.nan                        # Air pressure-induced line shift
    iso_csv_df['V_i'] = list(filter(None, upper_global_quanta))            # Upper-state 'global' quanta
    iso_csv_df['V_f'] = list(filter(None, lower_global_quanta))            # Lower-state 'global' quanta
    iso_csv_df['Q_i'] = list(filter(None, upper_local_quanta))             # Upper-state 'local' quanta
    iso_csv_df['Q_f'] = list(filter(None, lower_local_quanta))             # Lower-state 'local' quanta                  
    iso_csv_df['Iref'] = np.nan                        # Reference indices
    iso_csv_df['*'] = np.nan                           # Flag

    return iso_csv_df
    

In [15]:
trans_col_name = ['i', 'f', 'A_if']
t_df = dict()
species_csv_df = pd.DataFrame()
trans_filenames = glob.glob(read_path + path_mol_iso + '/' + '*trans.bz2')
for trans_filename in tqdm(trans_filenames):
    t_df[trans_filename] = pd.read_csv(trans_filename, compression='bz2', sep='\s+',
                                       usecols=[0,1,2], header=None, names=trans_col_name,
                                       chunksize=100_000_000, iterator=True, low_memory=False)

    iso_csv_df = pd.DataFrame()
    for trans_df in t_df[trans_filename]:
        unc_trans_df = extract_unc_trans(trans_df)
        if len(unc_trans_df) != 0:
            iso_csv_df = calculate_csv(unc_states_df, unc_trans_df)
            
    species_csv_df = species_csv_df.append(iso_csv_df)

order = [M_mol_iso, 'I', 'v', 'S', 'A', 'gm_a', 'gm_s', 'E_f', 'n_a', 'dt_a',
         'V_i', 'V_f', 'Q_i', 'Q_f', 'Ierr', 'Iref', '*', 'g_i', 'g_f']
species_csv_df = species_csv_df[order]
# Sort by increasing wavenumber
species_csv_df = species_csv_df.sort_values(['v'], ascending = True).reset_index(drop=True)

species_csv_df.to_csv(result_path + path_mol_iso.replace('/','__') + '.csv', header=True, index=False) 

In [16]:
species_csv_df

,M of NH3__14N-1H3__CoYuTe,I,v,S,A,gm_a,gm_s,E_f,n_a,dt_a,V_i,V_f,Q_i,Q_f,Ierr,Iref,*,g_i,g_f
0,11,1,0.006261,2.884718e-42,3.261200e-24,NaN,NaN,2314.701483,NaN,NaN,"0 0 0 1 0 1 a E""","0 0 0 1 0 1 a E""","E"" 9 8 E'","E' 8 3A2""",0.000294,NaN,NaN,114,102
1,11,1,0.013063,6.921067e-37,3.508500e-19,NaN,NaN,1885.948547,NaN,NaN,0 0 0 1 0 1 s E',0 2 0 0 0 0 sA1',"E"" 5 3A1""",E' 5 2 E',0.000332,NaN,NaN,66,66
2,11,1,0.016712,4.658101e-50,2.863000e-26,NaN,NaN,4751.536300,NaN,NaN,"0 1 1 0 1 0 a E""",1 1 0 0 0 0 sA1',"E' 6 5 E""","E"" 7 5 E""",0.000347,NaN,NaN,78,90
3,11,1,0.018070,8.071828e-40,1.691800e-11,NaN,NaN,6848.317501,NaN,NaN,"1 0 1 0 1 0 a E""",1 0 1 0 1 0 s E',"E"" 5 4 E'",E' 5 4 E',0.000395,NaN,NaN,66,66
4,11,1,0.018650,1.796015e-41,2.656800e-19,NaN,NaN,4040.303144,NaN,NaN,"0 0 1 0 1 0 a E""",1 0 0 0 0 0 sA1',"E' 9 9A1""","E"" 8 1 E""",0.000903,NaN,NaN,114,102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18788,11,1,100.001689,1.967811e-34,9.205500e-06,NaN,NaN,5151.114544,NaN,NaN,0 0 1 1 1 1 s E',"1 0 0 1 0 1 a E""","E"" 4 1 E""","E' 4 1 E""",0.000928,NaN,NaN,54,54
18789,11,1,100.017947,1.095006e-26,5.175500e-02,NaN,NaN,3442.231307,NaN,NaN,0 0 0 2 0 2 s E',0 0 0 2 0 2 s E',"A2"" 5 1 E""",A2' 4 2 E',0.000918,NaN,NaN,132,108
18790,11,1,100.030340,6.825591e-41,4.384500e-13,NaN,NaN,4847.660741,NaN,NaN,0 1 1 0 1 0 s E',"0 1 1 0 1 0 a E""",E' 7 2 E',"E"" 7 6A2'",0.000330,NaN,NaN,90,90
18791,11,1,100.050637,5.040322e-41,2.355300e-12,NaN,NaN,5150.765820,NaN,NaN,0 0 1 1 1 1 s E',1 0 0 1 0 1 s E',E' 4 0A1',"E"" 4 1 E""",0.000741,NaN,NaN,54,54
